In [2]:
import time
from selenium import webdriver
from bs4 import BeautifulSoup
import re
import pandas as pd

In [ ]:
import requests

brand = 'omega'

df = pd.read_csv(brand + '.csv')

for index in range(0,df.shape[0]):
    first_url = df.iloc[index , 1]
    brand = df.iloc[index , 0]
    brand = str(brand)
        
    price = df.iloc[index , 2]
    
    index = str(index)
    price =str(price)
    
    f = open('Data_Out/' + brand + '___' + index + '___' + price +  '.png' ,'wb')
    f.write(requests.get(first_url).content)
    f.close()
    print('downloading image' + index)

downloading image0
downloading image1
downloading image2
downloading image3
downloading image4
downloading image5
downloading image6
downloading image7
downloading image8
downloading image9
downloading image10
downloading image11
downloading image12
downloading image13


137
